# Pointillism effect in Python

In [ ]:
import collections

import numpy as np


class BrushstrokeNode:
    def __init__(self, img_source: np.ndarray, radius: int, position: tuple[int, int], sub_matrix_size = 1):
        self.__img_source = img_source
        self.radius = radius
        self.position = position
        self.fill_color = (0, 0, 0, 0)
        self.sub_matrix_size = sub_matrix_size

        self.update()

    def update(self):
        (x, y), r = self.position, self.radius
        sub_m = self.__img_source[y:y + self.sub_matrix_size, x: x + self.sub_matrix_size]
        media_color = np.median(sub_m, axis=0).astype(int)[0]

        self.fill_color = (*media_color, self.alpha)

    @property
    def alpha(self):
        return self.fill_color[3]

    @property
    def rgb(self) -> tuple[int, int, int]:
        return self.fill_color[0:3]

    @property
    def xyr(self):
        (x, y), r = self.position, self.radius
        return x - r, y - r, x + r, y + r

    def draw(self, canvas):
        canvas.ellipse(self.xyr, fill=self.rgb)

        return canvas


In [ ]:
import random

from PIL import Image, ImageDraw


def pointillism_effect(img: Image, n_points: int, min_alpha: int = 250, point_size_range: tuple[int, int] = (1, 5),
                       sub_matrix_size: int = 5) -> Image:
    m_img_source = np.array(img)
    img_canvas = Image.new("RGBA", size=img_source.size, color="white")
    canvas = ImageDraw.Draw(img_canvas)

    for _ in range(n_points):
        r = random.randint(*point_size_range)
        x, y = random.randint(0, img_canvas.width - 1), random.randint(0, img_canvas.height - 1)

        sub_m = m_img_source[y:y + sub_matrix_size, x: x + sub_matrix_size]
        media_color = np.median(sub_m, axis=0).astype(int)[0]

        alpha = random.randint(min_alpha, 255)
        fill_color = (*media_color, alpha)
        xyr = x - r, y - r, x + r, y + r

        canvas.ellipse(xyr, fill=fill_color)

    return img_canvas


def pointillism_effect_nodes(img: Image, n_points: int, point_size_range: tuple[int, int] = (1, 5),
                       sub_matrix_size: int = 5) -> Image:
    m_img_source = np.array(img)

    nodes = collections.deque()
    width, height = img_source.size

    for _ in range(n_points):
        r = random.randint(*point_size_range)
        position = random.randint(0, width - 1), random.randint(0, height - 1)

        node = BrushstrokeNode(m_img_source, r, position)
        node.sub_matrix_size = sub_matrix_size

        nodes.append(node)

    return nodes


def draw_nodes(img: Image, nodes: list[BrushstrokeNode]):
    img_canvas = Image.new("RGB", size=img_source.size, color="white")
    canvas = ImageDraw.Draw(img_canvas)

    for node in nodes:
        node.draw(canvas)

    return img_canvas



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def plot_rgb_plt(rgb_list):
    rgb = np.array(rgb_list, dtype=int)
    x, y, z = rgb[:, 0], rgb[:, 1], rgb[:, 2]

    colors = rgb / 255.0

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c=colors, s=50)

    ax.set_xlabel('R')
    ax.set_ylabel('G')
    ax.set_zlabel('B')

    return fig, ax


def plot_rgb_go(rgb_list):
    rgb = np.array(rgb_list, dtype=int)
    x, y, z = rgb[:, 0], rgb[:, 1], rgb[:, 2]

    colors = [f'rgb({r},{g},{b})' for r, g, b in rgb]

    fig = go.Figure(data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color=colors)
        )
    ])

    fig.update_layout(
        scene=dict(
            xaxis_title='R',
            yaxis_title='G',
            zaxis_title='B'
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    return fig


## Load image

In [ ]:
FILENAME = "resources/source-6.png"

img_source = Image.open(FILENAME).convert("RGB")
img_source

## Filter application

In [ ]:
nodes = pointillism_effect_nodes(img_source, 25000, sub_matrix_size=1, point_size_range=(5, 15))

img_result = draw_nodes(img_source, nodes)
img_result

In [ ]:
rgb = np.array([node.rgb for node in nodes])

fig, ax = plot_rgb_plt(rgb)